Copyright 2024 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

---

In [1]:
# %pip install git+https://github.com/google-deepmind/recurrentgemma.git

In [2]:
# %pip install torch kagglehub 

In [3]:
# pip install torch==2.2.1

In [14]:
import os
import kaggle
import json
# Replace 'path/to/kaggle.json' with your actual file path
config = {"username":"huzaifaghori","key":"c08f2b54aa777c1bd232e5cfc7b66ea8"}

api = kaggle.api
# Now you can use the Kaggle API with 'api' object


In [17]:
api._load_config(config_data=config)

In [18]:
#@title Imports
import pathlib
import kagglehub
import torch
import sentencepiece as spm
from recurrentgemma import torch as recurrentgemma

device = "cuda" if torch.cuda.is_available() else "cpu"

In [24]:
VARIANT = '2b-it' # @param ['2b', '2b-it'] {type:"string"}
weights_dir = kagglehub.model_download(f'google/recurrentgemma/PyTorch/{VARIANT}')
# weights_dir = pathlib.Path(f"/input/recurrentgemma/pytorch/{VARIANT}/1")


In [32]:

ckpt_path = f"{weights_dir}/{VARIANT}.pt"
vocab_path = f"{weights_dir}/tokenizer.model"

In [33]:
# Load parameters
params = torch.load(str(ckpt_path))
params = {k : v.to(device=device) for k, v in params.items()}

In [34]:
model_config = recurrentgemma.GriffinConfig.from_torch_params(params)
model = recurrentgemma.Griffin(model_config, device=device, dtype=torch.bfloat16)
model.load_state_dict(params)

<All keys matched successfully>

In [35]:
vocab = spm.SentencePieceProcessor()
vocab.Load(str(vocab_path))

True

In [36]:
sampler = recurrentgemma.Sampler(model=model, vocab=vocab)

In [37]:
input_batch = [
  "who is the president of USA?",
    "?"
]

# 300 generation steps
out_data = sampler(input_strings=input_batch, total_generation_steps=300)
print(out_data.text)
for input_string, out_string in zip(input_batch, out_data.text):
  print(f"Prompt:\n{input_string}\nOutput:\n{out_string}")
  print(10*'#')

KeyboardInterrupt: 